# Sentinel-2 Sunglint Finder with Earth Engine Python API

This notebook shows how to find Sentinel-2 scenes over sunglint geometry using the Earth Engine Python API.  
Adjust the `bbox`, `start_date`, `end_date`, and other parameters as needed.


In [ ]:
# 1. Install and import Earth Engine Python API
# Uncomment the following line if running in Colab or a new environment:
# !pip install earthengine-api

import ee
import math
import geopandas as gpd
from shapely import Point
import geemap

# Initialize the Earth Engine client. 
# If running for the first time, you may need to authenticate:
# ee.Authenticate()
ee.Initialize()
print('Earth Engine initialized.')


In [ ]:
# West Africa Tropical Atlantic (off Senegal to Angola)
aoi_west_africa = ee.Geometry.Rectangle([-30.0, 0.0, -10.0, 15.0])

# Equatorial Atlantic (Gulf of Guinea)
aoi_equatorial_atlantic = ee.Geometry.Rectangle([-10.0, -5.0, 10.0, 10.0])

# Eastern Pacific (off Central America)
aoi_eastern_pacific = ee.Geometry.Rectangle([-110.0, -5.0, -90.0, 10.0])

# Western Indian Ocean (near Madagascar and Mozambique Channel)
aoi_indian_ocean = ee.Geometry.Rectangle([35.0, -20.0, 60.0, 0.0])

# Equatorial Pacific (near Kiribati, remote tropical waters)
aoi_central_pacific = ee.Geometry.Rectangle([-180.0, -10.0, -160.0, 10.0])

# Bay of Bengal (Eastern Indian Ocean, often glinty)
aoi_bay_of_bengal = ee.Geometry.Rectangle([80.0, 5.0, 95.0, 20.0])

# South China Sea (tropical glint-prone waters)
aoi_south_china_sea = ee.Geometry.Rectangle([110.0, 0.0, 125.0, 15.0])


brend_example = -91.469444, 28.430278
brend_example_date = "5-15-2022"

In [ ]:
import pandas as pd
# infrastructure_points = ee.FeatureCollection("projects/ee-brendan-skytruth/assets/gfw_fixedInfra_102024")
# infrastructure_points.filter(ee.Filter.eq('label', 'oil'))
# infra_ls = infrastructure_points.toList(1000)

infra = pd.read_csv(r"C:\Users\ebeva\SkyTruth\methane\nonoise_SAR_fixed_infrastructure.csv")

# 1. Define the Gulf of Mexico bounding box
min_lon, min_lat = -98.0, 24.0
max_lon, max_lat = -80.5, 31.0


In [ ]:
# Convert infra DataFrame to a GeoDataFrame and filter to Gulf of Mexico bounding box
gdf = gpd.GeoDataFrame(
    infra,
    geometry=gpd.points_from_xy(infra['lon'], infra['lat']),
    crs="EPSG:4326"
)

gulf_bbox = (min_lon, min_lat, max_lon, max_lat)
gdf_gulf = gdf.cx[min_lon:max_lon, min_lat:max_lat]
gdf_gulf

In [ ]:
coords = [
    [102.98827, 7.59247],
    [102.98800, 7.59238],
    [102.98827, 7.59283],
    [102.98799, 7.59346],
    [102.98827, 7.59265]
]

# Create a MultiPoint geometry
stationary_infra = ee.Geometry.MultiPoint(coords)

In [ ]:
from pysolar.solar import get_altitude, get_azimuth
from datetime import datetime, timezone

def expanded_mosaic(ref_img):
    ref_time = ee.Date(ref_img.get('system:time_start'))

    # Step 3: Filter all S2_SR tiles within ±1 min of this timestamp
    collection = ee.ImageCollection('COPERNICUS/S2_SR') \
        .filterDate(ref_time.advance(-1, 'minute'), ref_time.advance(1, 'minute'))
    # Step 4: Mosaic
    return collection.mosaic()

def print_s2_metadata_angles(img):
    """
    Prints Sentinel-2 solar angles from metadata + Pysolar estimate,
    and assumes nadir (0° zenith) satellite view for simplicity.
    """
    # 1. Get image center and time
    centroid = img.geometry().centroid()
    coords = centroid.getInfo()['coordinates']
    lon, lat = coords[0], coords[1]
    
    timestamp_ms = img.get('system:time_start').getInfo()
    dt = datetime.fromtimestamp(timestamp_ms / 1000, tz=timezone.utc)

    # 2. Read metadata properties (these are scalar per image)
    solar_zenith_img = img.get('MEAN_SOLAR_ZENITH_ANGLE').getInfo()
    solar_azimuth_img = img.get('MEAN_SOLAR_AZIMUTH_ANGLE').getInfo()

    # 3. Estimate sun angles with pysolar
    solar_altitude = get_altitude(lat, lon, dt)
    solar_zenith_py = 90.0 - solar_altitude
    solar_azimuth_py = get_azimuth(lat, lon, dt)

    # 4. Print results
    print(f"\n🛰️  Sentinel-2 Image ID: {img.id().getInfo()}")
    print(f"🕒  Acquisition time (UTC): {dt.isoformat()}")
    print(f"📍  Location (lat, lon): ({lat:.4f}, {lon:.4f})\n")

    print("📡 Satellite viewing angles (assumed):")
    print(f"   - Zenith:  0.00°  (nadir)")
    print(f"   - Azimuth: unknown\n")

    print("☀️  Solar angles (from metadata):")
    print(f"   - Zenith:  {solar_zenith_img:.2f}°")
    print(f"   - Azimuth: {solar_azimuth_img:.2f}°\n")

    print("🔆 Solar angles (computed by Pysolar):")
    print(f"   - Zenith:  {solar_zenith_py:.2f}°")
    print(f"   - Azimuth: {solar_azimuth_py:.2f}°")

In [ ]:
# Step 3: Define functions used for creating plume maps.

def linear_fit(img):
    """
    Perform a linear regression of B11 vs. B12 in the AOI to estimate coefficient c_fit.
    Sets the coefficient and coefficient list as properties on the image.
    """
    # Prepare the regression: stack B12 (x) and B11 (y).
    x_var = img.select("B12")
    y_var = img.select("B11")
    stacked = ee.Image.cat([x_var, y_var])

    # Run a linear regression reducer with numX=1, numY=1.
    fit_dict = stacked.reduceRegion(
        reducer=ee.Reducer.linearRegression(numX=1, numY=1),
        # geometry=aoi,
        scale=20,
        bestEffort=True,
    )
    # The result is a dictionary containing an “array” of coefficients.
    coeff_array = ee.Array(fit_dict.get("coefficients"))
    coeff_list = coeff_array.toList().get(0)
    c0 = ee.List(coeff_list).get(0)

    return img.set({"c_fit": c0, "coef_list": coeff_list})


def MBSP(img):
    """
    Calculate the MBSP ratio (IR_comp) per pixel:
        IR_comp = ((c * B12) - B11) / B11
    where c is the regression coefficient stored in the image properties.
    """
    c = ee.Number(img.get("c_fit"))
    formula = (
        ee.Image(img.select("B12")).multiply(c)
        .subtract(img.select("B11"))
        .divide(img.select("B11"))
    ).rename("IR_comp")
    return img.addBands(formula)


def mask_land(image):
    """
    Use JAXA/GCOM-C LST as a land mask. We take the mean over Jan 2024,
    then mask out any pixel where LST_AVE is non-zero (i.e., keep non-land).
    """
    # Filter the land surface temperature collection to January 2024.
    lst_collection = ee.ImageCollection("JAXA/GCOM-C/L3/LAND/LST/V3").filterDate(
        "2024-01-01", "2024-02-01"
    )
    lst_mosaic = lst_collection.mean()
    # Any pixel with a non-zero LST_AVE is considered land.
    land_masker = lst_mosaic.select("LST_AVE").reduce(ee.Reducer.anyNonZero())

    # Keep only pixels where land_masker == 0 (i.e., water or non-land).
    return image.updateMask(land_masker.unmask(0).eq(0))

In [ ]:

# infra.sample(1)
row = gdf_gulf.sample(1).iloc[0]
stationary_infra = ee.Geometry.Point([row['lon'], row['lat']])
Map = geemap.Map()
Map.addLayer(stationary_infra)
Map.centerObject(stationary_infra, 8)
Map

In [ ]:

# Define a wide sunglint-prone AOI (off West Africa, open tropical ocean)
# aoi = ee.Geometry.Rectangle([-30.0, 0.0, -10.0, 15.0])

# 1. Filter Sentinel-2 SR Harmonized collection



s2 = ee.ImageCollection('COPERNICUS/S2_SR') \
    .filterDate('2018-01-01', '2025-12-01') \
    .filterBounds(stationary_infra) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
    # .filter(ee.Filter.gt('MEAN_SOLAR_ZENITH_ANGLE', 60))  # Stronger glint
# 2. Apply cloud mask to s2
def mask_s2_clouds(image):
  """Masks clouds in a Sentinel-2 image using the QA band.

  Args:
      image (ee.Image): A Sentinel-2 image.

  Returns:
      ee.Image: A cloud-masked Sentinel-2 image.
  """
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloud_bit_mask = 1 << 10
  cirrus_bit_mask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = (
      qa.bitwiseAnd(cloud_bit_mask)
      .eq(0)
      .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
  )

  return image.updateMask(mask)#.divide(10000)
# s2 = s2.map(mask_s2_clouds)
s2 = s2.map(mask_land)
s2.size().getInfo()

In [ ]:


# # 2. Add optional Sun Glint Index (green vs SWIR)
# def add_sgi(img):
#     green = img.select('B3').divide(10000)
#     swir = img.select('B11').divide(10000)
#     sgi = green.subtract(swir).divide(green.add(swir)).rename('SGI')
#     return img.addBands(sgi)

# s2 = s2.map(add_sgi)

def addGlintIndex(img):
    nir = img.select('B8A')
    green = img.select('B3')
    gi = nir.subtract(green).divide(nir.add(green)).rename('GlintIndex')
    return img.addBands(gi)

s2 = s2.map(addGlintIndex)

# 3. Estimate sunglint pixel fraction from SWIR reflectance (BOEM method)
def add_b11_glint_fraction(img):
    # swir = img.select('B11').divide(10000)
    # glint_mask = swir.gt(0.23)  # SWIR glint threshold
    swir = img.select('B11').divide(10000)
    
    # 2. Build a mask for 0.23 < SWIR < 0.40
    var_low  = swir.gt(0.13)
    var_high = img.select("MSK_CLDPRB").lt(5)
    # Combine them:
    glint_mask = var_low.And(var_high).rename('glintMask').unmask(0)

    stats = glint_mask.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=img.geometry(),
        scale=100,
        bestEffort=True,
        maxPixels=1e13
    )
    glint_fraction = stats.get('glintMask')
    return img.set('b11_glint_fraction', glint_fraction)

def add_glint_index_fraction(img):
    # swir = img.select('B11').divide(10000)
    # glint_mask = swir.gt(0.23)  # SWIR glint threshold
    gi = img.select('GlintIndex')
    
    # 2. Build a mask for 0.23 < SWIR < 0.40
    var_low  = gi.gt(0.0)
    var_high = img.select("MSK_CLDPRB").lt(5)
    # Combine them:
    glint_mask = var_low.And(var_high).rename('glintIndexMask').unmask(0)

    stats = glint_mask.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=img.geometry(),
        scale=100,
        bestEffort=True,
        maxPixels=1e13
    )
    glint_fraction = stats.get('glintIndexMask')
    return img.set('glint_index_fraction', glint_fraction)

s2 = s2.map(add_glint_index_fraction)


In [ ]:
# sorted_s2 = s2.sort('b11_glint_fraction', False).toList(10)
# sorted_s2 = s2.sort('glint_index_fraction', False).toList(10)
sorted_s2 = s2.sort('MEAN_SOLAR_ZENITH_ANGLE').toList(10)
idx = 2
high_glint_fract = ee.Image(sorted_s2.get(idx))

In [ ]:
print_s2_metadata_angles(high_glint_fract)

In [ ]:
high_glint_fract.get('glint_index_fraction').getInfo()

In [ ]:
high_glint_fract.get('b11_glint_fraction')

In [ ]:
# Apply linear_fit, then MBSP to high_glint_fract
high_glint_fract_with_mbsp = MBSP(linear_fit(high_glint_fract))

In [ ]:
Map = geemap.Map(center=[7.5, -20], zoom=5)
# Map.addLayer(s2.median().select('SGI'), {'min': -1, 'max': 1, 'palette': ['blue', 'white', 'red']}, 'SGI')
# Map.addLayer(s2.median().visualize(bands=['B4', 'B3', 'B2'], min=0, max=3000), {}, 'RGB')
# Map

reference_id1 = 'COPERNICUS/S2_SR/20230611T162839_20230611T164034_T16RBT'
full_glint = expanded_mosaic(high_glint_fract)
# full_glint = expanded_mosaic(ee.Image(reference_id1))

# Map.addLayer(high_glint_fract, {}, 'full_scene', False)
# Map.addLayer(high_glint_fract.select("GlintIndex"), {'min': -1, 'max': 1, 'palette': ['blue', 'white', 'red']}, "GlintIndex")
# Map.addLayer(high_sgi.visualize(bands=['B4', 'B3', 'B2'], min=0, max=5000), {}, 'high SGI glint')
# Map.addLayer(add_sgi(full_glint).select('SGI'), {'min': -1, 'max': 1, 'palette': ['blue', 'white', 'red']}, 'glint SGI')
Map.addLayer(addGlintIndex(full_glint).select('GlintIndex'), {'min': -.6, 'max': .6, 'palette': ['blue', 'white', 'red']}, 'glint Index')
# Map.addLayer(high_glint_fract_with_mbsp.select("IR_comp"), {'min': -0.5, 'max': 0.5, 'palette': ['blue', 'white', 'red']}, 'MBSP IR_comp')
Map.addLayer(full_glint.visualize(bands=['B4', 'B3', 'B2'], min=0, max=4000), {}, 'High Glint S2')


Map.addLayer(stationary_infra,{'color': 'red'}, 'Stationary')
Map.centerObject(stationary_infra, 9)
Map

In [ ]:
drawn_feat = Map.draw_last_feature

In [ ]:
from Shapely import shape
drawn_feat
geojson_geom = geemap.ee_to_geojson(drawn_feat)
shapely_geom = shape(geojson_geom['geometry'])

In [ ]:
high_glint_fract